##Simple Linear Regression

In [ ]:
import pandas as pd
df=pd.read_csv('toyota.csv')
df.head()

In [ ]:
#Let us look at the correlation between various variables in the toyota dataframe
df.corr()

###The correlation matrix displays correlation coefficients for all pairs of variables.
###The correlation coefficient shows how strongly two variables. It can be +ve(increasing effect) or -ve(decreasing effect).
###We saw that there is a negative correlation between price of a car and its age i.e. as age increases the price decreases. 

In [ ]:
#To visualize the correlation
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.regplot(x=df['Age'],y=df['Price'],fit_reg=True,color='green')
plt.show()

###A regression model describes how y(Price) is related to x(Age). A simple linear regression model is of the form
###y=b0+b1*x
###The above equation represents a straight line whose slope is b1 and y intercept is b0. Thus the equation helps us to predict values of y corresponding to some value of x. 

In [ ]:
#Let us look how we can get a linear regression model for x=Age and y=Price
import statsmodels.api as s
x=list(df['Age'])
y=list(df['Price'])
x=s.add_constant(x)
model=s.OLS(y,x)
result=model.fit()
result.summary()

###The coef column in the above result contains y intercept(const) and the slope(x1). Besides we can see various other terms like R^2, F-statistic, t- statistic etc.
###These terms are useful when we do not consider the entire population but only the samples. In this case, the linear regression model obtained is called estimated lrm. 
###To check if the estimated slope holds good for the entire population we perform t test(for two variables) or f test(anova)(generally used when there are more than two variables).
###The null hypothesis is:H0:B0=0 where B0 is the slope corresponding to the population.
###If the null hypothesis becomes true, then it means that there is no relation between y and x at the population level.
###The term R^2 is called coefficient of determination. It describes how well the model performs. It basically checks if the model's error is explained by variation of x alone or not. If R^2 is 1 it means the model is perfect i.e. it gives exact values of y corresponding to x. If R^2 is zero, it means that there is no relation between x and y

In [ ]:
#Let us perform the same operation using sklearn module
import sklearn.model_selection as sk
x=df['Age'].values.reshape(-1,1)     #Converting data into 2D arrays
y=df['Price'].values.reshape(-1,1)
x_train,x_test,y_train,y_test=sk.train_test_split(x,y,test_size=.2,random_state=88) 
#The data is split into two parts:train and test
#train is used to train the model while test is used to check the accuracy of the model. 

from sklearn.linear_model import LinearRegression
reg=LinearRegression()
reg.fit(x_train,y_train) #Prepare the model
#Display information
print("Y Intercept=",reg.intercept_)
print("Slope=",reg.coef_)
print("Score(R^2)=",reg.score(x_test,y_test))
y_predict=reg.predict(x_test)
print("x value\tPredicted y value\tActual y value")
for i in range(10):
  print(x_test[i],"\t",y_predict[i],"\t",y_test[i])

In [ ]:
#Now suppose we consider only samples instead of the populations.
#Let the sample size be 200
sns.regplot(x=df['Age'][:200],y=df['Price'][:200],fit_reg=True,color='green')
plt.scatter(np.mean(df['Age'][:200]),np.mean(df['Price'][:200]),color='red')  #This line shows the mean of the data as red dots
plt.show()
#Notice the light colored boundary around the regression line. 
#This is a Confidence interval for the mean value of y for a given value of x
#This applies when we look at samples instead of entire populations

In [ ]:
#Prepare model for sample
import random as rd
x=rd.sample(list(df['Age']),200)
y=rd.sample(list(df['Price']),200)
x1=x.copy()
y1=y.copy()
x=s.add_constant(x)
model=s.OLS(y,x)
result=model.fit()

#Preparing prediction intervals, confidence intervals
from statsmodels.stats.outliers_influence import summary_table
st,data1,ss2=summary_table(result,alpha=0.05)
fittedvalues=data1[:,2]
ci_low,ci_up=data1[:,4:6].T
pred_low,pred_up=data1[:,6:8].T

#Displaying the graph with prediction and confidence intervals.
X=s.add_constant(x)
fig,ax=plt.subplots(figsize=(8,6))
ax.plot(x,y,'o',label='df')
ax.plot(X,fittedvalues,'r-',label='OLS')
ax.plot(X,ci_low,'b--',label="Conf. Int")
ax.plot(X,ci_up,'b--',label='Conf. Int')
ax.plot(X,pred_low,'g--',label="Pred. Int")
ax.plot(X,pred_up,'g--',label="Pred. Int")
ax.legend(loc='best');
plt.show()

###Residual Analysis:To determine if the assumptions that we make about the error term in the linear regression model are true. The assumptions are:
###1.Normal distribution
###2.Mean is Zero
###3.Independent and
###4.Variance is same for all x terms
###We must check the validity of these assumptions because they are the ground for the t test and f test which were discussed above. If the assumptions are questionable, then the test itself is compromised.
###We make use of residual(error) plots to determine the validity of the assumptions.

In [ ]:
#Residual Plot of Residual against x
sns.residplot(x=x1,y=y1)
plt.show()
#If the pattern is more or less rectangular,it indicates the variance of error is nearly same for all x values
#Else variance is not same.

#The residual plot of residual against y^ is also similar but more widely used because, 
#in the case of multiple regression, we have to draw seperate plots for each variable. This can be avoided by
#using a single plot of residual against y^

In [ ]:
#Next we need to check if the residuals have a normal distribution or not.
#For that we use standardized residuals.
#If most of these(95%) residuals fall within -2 and 2, then we can say that residuals have normal dis.
#Otherwise they do not.
influence=result.get_influence()
resid_student=influence.resid_studentized_external
c=0
#finding what percentage of values lie between -2 and 2
for i in resid_student:
  if(i>-2 and i<2):
    c=c+1
plt.title("Percentage of Values between -2 and +2="+str((c/len(resid_student))*100))
plt.scatter(x1,resid_student)
plt.show()
#It can be observed that the two plots(residual and standard-residual) are exactly same but only the y axis has been standardized.

In [ ]:
#Another way to assess normailty is to use the mormal probability plot.
#In this plot the standardized residuals are compared with similar normal scores from a normal distribution.
#(where mean=0 and standard deviation is 1)
#If this plot is inclined at approximately 45 degrees to the x axis, then the residuals are said to be normally distributed.
from scipy import stats
import statsmodels.api as sm
res=result.resid
prbplot=sm.ProbPlot(res,stats.norm,fit=True)
fig=prbplot.qqplot(line='45')
plt.show()
